In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#파일 읽어오기

meta = pd.read_csv('/kaggle/input/the-movies-dataset/movies_metadata.csv')
meta

In [ ]:
#원하는 것만 가져오기

meta = meta[['id', 'original_title', 'original_language', 'genres',]]
meta

In [ ]:
#이름 재정의

meta = meta.rename(columns={'id':'movieId', 'original_title':'title', 'original_language':'language'})
meta

In [ ]:
#영어로 된 것만 남겨보기 (en) ~> 정제

meta = meta.loc[meta['language'] == 'en', :]
#meta.loc=['행이름', '열??']
meta

#컬럼 가져오기
#meta = meta.log[:, ['title', 'language']]
#meta = meta.ilog[:, 2]
#meta = meta.title
#txt ~> Vector, 표 ~> Matrix

In [ ]:
#meta.movieId를 Int형으로

meta.movieId = pd.to_numeric(meta.movieId)
meta.movieId

In [ ]:
#genre 정제하기

items = meta.loc[0, 'genres']
#eval ~> python코드를 txt로 가지고 있을때 python으로

txt_items = eval(items)
itemSet = set()

#genre에서 name만 출력해보기
for item in txt_items:
    itemSet.add(item['name'])

itemSet

#==print(set(item['name'] for item in eval(items)))

In [ ]:
#함수로 만들기

def str_to_set(x):
    genreSet = set()
    for item in eval(x):
        genreSet.add(item['name'])
    return genreSet

In [ ]:
#apply ~> 함수의 결과를 다시 데이터에 넣어줌

meta.genres = meta.genres.apply(str_to_set)
meta

In [ ]:
#keywords데이터로 읽기

keyword = pd.read_csv('/kaggle/input/the-movies-dataset/keywords.csv')
keyword

In [ ]:
#keyword의 집합으로 정제

keyword.keywords = keyword.keywords.apply(str_to_set)
keyword

In [ ]:
#keyword Id를 movieId로 변경
#DataType Int로 변경

keyword = keyword.rename(columns={'id':'movieId'})
keyword.movieId = pd.to_numeric(keyword.movieId)
keyword

In [ ]:
#같은 movieId를 갖는 Data 합치기(=join)
#merge(merge data1, data2, 어떤 기준으로?, 어떻게 merge)
#inner ~> 공통된 기준이 있는 부분만 남김
#outer ~> 공통 X더라도 join

meta = pd.merge(meta, keyword, on = 'movieId', how = 'inner')
meta

In [ ]:
#영화를 찾아서 합치고 출력

dk = meta.loc[meta.title == 'The Dark Knight'].iloc[0]
#dk
dkr = meta.loc[meta.title == 'The Dark Knight Rises'].iloc[0]
pd.concat([dk, dkr], axis=1).T

In [ ]:
#Jacard Similarity def 구현
#|교집합|/|합집합|

def jacard_similarity(s1, s2):
    if (len(s1|s2) == 0):
        return 0
    return (len(s1&s2)/len(s1|s2))

jacard_similarity(dk.genres|dk.keywords, dkr.genres|dkr.keywords)

In [ ]:
#Rating 데이터 읽고 정제

ratings = pd.read_csv('/kaggle/input/the-movies-dataset/ratings_small.csv')
ratings

In [ ]:
ratings.movieId #int64 이므로 int로 변경 안해도 O

In [ ]:
#Merge로 title 가져오기

ratings = pd.merge(ratings, meta[['movieId', 'title']], on='movieId', how='inner')
ratings

In [ ]:
#pivot Table 만들기(Pivot은 비효율적 ~> 잘 안씀)
#user ~가 (row부분) title(column)에 대해 ??점(value)를 주었다!

pivotRating = ratings.pivot_table(index='userId', columns='title', values='rating')
pivotRating

In [ ]:
#user1의 value
pivotRating.loc[1,:]

#user1과 user2의 Pearson Correlation Coefficient
def pearson_similarity(u1, u2):#평균으로 뺀 후에 cos
    u1_c = u1 - u1.mean()
    u2_c = u2 - u2.mean()
    denom = np.sqrt(np.sum(u1_c ** 2) * np.sum(u2_c ** 2))
    if denom != 0:
        return np.sum((u1_c)*(u2_c))/denom
    else:
        return 0

pearson_similarity(pivotRating.loc[1,:], pivotRating.loc[2,:])

In [ ]:
#TEST

dkU = pivotRating['The Dark Knight']
pnU = pivotRating['Prom Night']
yiU = pivotRating['Young and Innocent']
dkpnPS = pearson_similarity(dkU, pnU)
dkyiPS = pearson_similarity(dkU, yiU)

#dkpnPS
dkyiPS

In [ ]:
#영화 하나를 골라서 가장 유사한 영화들을 뽑아보기

def find_similarity_movies(input_title, matrix, n, alpha):
    input_meta = meta.loc[meta['title']==input_title].iloc[0]
    input_set = input_meta.genres | input_meta.keywords
    
    result = []
    
    for this_title in pivotRating.columns:
        if this_title == input_title:
            continue
        
        this_meta = meta.loc[meta['title']==this_title].iloc[0]
        this_set = this_meta.genres | this_meta.keywords
        
        thisinputPS = pearson_similarity(pivotRating[this_title], pivotRating[input_title])
        thisinputJac = jacard_similarity(this_set, input_set)
    
        score = alpha * thisinputPS + (1 - alpha) * thisinputJac
        result.append((this_title, thisinputPS, thisinputJac, score))
        
    result.sort(key = lambda r: r[3], reverse=True)
    return result[:n]
    
result = find_similarity_movies('The Dark Knight', pivotRating, 10, 0.3)
pd.DataFrame(result, columns=['title', 'pearson', 'jaccard', 'score'])
